In [1]:
from pathlib import Path
import os
import yaml
from openslide import OpenSlide
import numpy as np
import cv2
from src.utils import *
from PIL import Image

from matplotlib import pyplot as plt

In [3]:
config = load_config()
config

{'dataset_path': '/Users/cmkim/workspace/datasets/TIGER/'}

In [ ]:
dataset_path = Path(config["dataset_path"])
dataset_path

In [ ]:
# subset_path = dataset_path / "wsirois/wsi-level-annotations/images"
subset_path = dataset_path / "wsibulk/images"

subset_path

In [ ]:
# wsi_path = subset_path / os.listdir(subset_path)[0]
# wsi_path = subset_path / "111S.tif"
# wsi_path = subset_path / "103S.tif"
wsi_path = subset_path / "119S.tif"

wsi_path

In [ ]:
slide = OpenSlide(filename=wsi_path)
slide

In [ ]:
slide.properties["openslide.mpp-x"]

In [ ]:
level_d = slide.level_dimensions

In [ ]:
level = slide.get_best_level_for_downsample(32)

In [ ]:
image = np.array(
    slide.read_region(
        location=(0, 0),
        level=level, 
        size=level_d[level]
    )
)

In [ ]:
if image.shape[-1] == 4:
    image = rgba2rgb(image)

In [ ]:
# gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
ret, thresh = cv2.threshold(
    src=gray, 
    thresh=-1, 
    maxval=255, 
    type=cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU
    # type=cv2.THRESH_OTSU
)

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))

# np.max([32 / mpp_size])

In [ ]:
gray_morphology = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

In [ ]:
contours, hierarchy = cv2.findContours(
    image=gray_morphology, 
    mode=cv2.RETR_EXTERNAL,
    # mode=cv2.RETR_CCOMP, 
    method=cv2.CHAIN_APPROX_SIMPLE
)

In [ ]:
image_contours = image.copy()
_ = cv2.drawContours(
    image=image_contours, 
    contours=contours, 
    contourIdx=-1, 
    color=(0, 255, 0), 
    thickness=3
)

In [ ]:
Image.fromarray(np.hstack((image, image_contours))).show()

In [ ]:
max_size = int(np.sqrt(np.max([cv2.contourArea(_) for _ in contours])) * 0.2)
max_size = max(3, min(max_size, 100))

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (max_size, max_size))

In [ ]:
gray_morphology = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

In [ ]:
contours, hierarchy = cv2.findContours(
    image=gray_morphology, 
    mode=cv2.RETR_EXTERNAL,
    # mode=cv2.RETR_CCOMP, 
    method=cv2.CHAIN_APPROX_SIMPLE
)

In [ ]:
image_contours = image.copy()
_ = cv2.drawContours(
    image=image_contours, 
    contours=contours, 
    contourIdx=-1, 
    color=(0, 255, 0), 
    thickness=3
)

In [ ]:
Image.fromarray(np.hstack((image, image_contours))).show()